In [1]:
# things we need for NLP
# NLTK is a leading platform for building Python programs to work with human language data.
# It provides easy-to-use interfaces to over 50 corpora and lexical resources such as WordNet,
# along with a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing,
# and semantic reasoning.
import nltk
# A computer program or subroutine that stems word may be called a stemming program, stemming algorithm, or stemmer.
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
# things we need for Tensorflow
# Numpy is a library for the Python programming language, adding 
# support for large, multi-dimensional arrays and matrices, along with a 
# large collection of high-level mathematical functions to operate on these arrays.
import numpy as np
# TFlearn is a modular and transparent deep learning library built on top of Tensorflow.
# It was designed to provide a higher-level API to TensorFlow in order to facilitate and speed-up experimentations,
# while remaining fully transparent and compatible with it.
import tflearn
# Tensorflow is an open source machine learning library, which is used to develop and train ML models
import tensorflow as tf
import random

W0705 16:53:40.451355   244 deprecation_wrapper.py:119] From c:\program files\python37\lib\site-packages\tflearn\helpers\summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W0705 16:53:40.454747   244 deprecation_wrapper.py:119] From c:\program files\python37\lib\site-packages\tflearn\helpers\trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0705 16:53:40.467782   244 deprecation_wrapper.py:119] From c:\program files\python37\lib\site-packages\tflearn\collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0705 16:53:40.475803   244 deprecation_wrapper.py:119] From c:\program files\python37\lib\site-packages\tflearn\config.py:123: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W0705 16:53:40.480257   244 deprecation_wrapper.py:119] From c:\program files\python37\lib\site-packages\t

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
# import our chat-bot intents file
import json
intent_file= open('intents.json','r')
intents = json.load(intent_file)
intent_file.close()

In [3]:
words = []
classes = []
references = []
words_to_ignore = ['?','!']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    #note arr['name'] is visiting that array by name 
    for pattern in intent['question_patterns']:
        # add to list containing the classes of words
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
        # tokenize each word in the sentence
#Tokenization is the act of breaking up a sequence of strings into pieces such as words, keywords, phrases, symbols and other elements called tokens
        word = nltk.word_tokenize(pattern)
        # append word to our words list
        words.extend(word)
        # add to documents in our corpus 
#         A corpus is a collection of machine-readable texts that have been produced in a natural communicative setting
#in the references multidim list of bot, append a list of words at index 0 and a list of contents of intents with key 'tag'
        references.append((word, intent['tag']))
        

# stem and lower each word and remove duplicates
words = [ stemmer.stem(word.lower()) for word in words if word not in words_to_ignore]
#set method removes the duplicates in the words and classes obtained
words = list(set(words))
classes = list(set(classes))

print (len(references), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

121 documents
26 classes ['soft_issues_response_errors', 'payments', 'goodbye', 'hours', 'bug_per', 'Ebanking', 'e_trans', 'greeting', 'services', 'EntSolutions', 'fins_trans', 'soft_issues_response_hard', 'finfinancials', 'soft_issues_response_soft_antivirus', 'thanks', 'ent_trans', 'soft_issues_response_noantivirus', 'opentoday', 'clearing_trans', 'get_services', 'departments', 'bug_resolve_affirmed', 'soft_issues', 'transaction', 'payments_n_clearing', 'get_info']
103 unique stemmed words ['ebank', 'bye', 'bug', 'explain', 'hav', 'inform', 'is', 'neg', 'how', 'op', 'mor', 'softw', 'tel', 'provid', 'problem', 'card', 'do', 'cash', 'ther', 'er', 'yo', 'much', 'what', 'hi', 'structure', 'nop', 'about', 'program', 'solv', 'who', 'good', 'the', 'depart', 'hard', 'up', 'i', 'see', 'sort', 'exceiv', 'a', 'when', 'enterpr', 'expery', 'an', 'let', 'opt', ',', 'serv', 'credit', 'ok', 'deal', 'acceiv', 'no', 'in', 'finfin', 'yep', 'mastercard', 'posit', 'goodby', 'field', 'off', 'ar', 'fintech

In [4]:
# create our training data array bags
training = []
output = []
# create an empty array for our output
null_output = [0] * len(classes)
#A vocabulary is a set of all words known to our bot
# training set, vocabulary of words for each sentence
#In the following method, we will convert the list of words we have to an array of tensors that tensorflow can use
for ref in references:
    # initialize our vocabulary of words
    vocab = []
    # list of tokenized words for the pattern
    pattern_words = ref[0]
    # stem each word, to its root(s) :-)
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our vocabulary of words array
    for w in words:
        vocab.append(1) if w in pattern_words else vocab.append(0)

    # output is a '0' for inactive and '1' for active tag
    output_row = list(null_output)
    output_row[classes.index(ref[1])] = 1
#append to multidim array training the vocabulary of root forms and the row they each correspond to
    training.append([vocab, output_row])

# shuffle our features and turn into np.array
#shuffling will reduce the variance and overfitting
random.shuffle(training)
training = np.array(training)

# create train and test lists
#the : stands for everything from the beginning to end , 0 represents the dimension, the 1st in 1d array
train_x = list(training[:,0])
train_y = list(training[:,1])
#print(train_x)

In [5]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
#shape will be 2D array of 1*length of a single train list
input_data = tflearn.input_data(shape=[None, len(train_x[0])])
#create fully connected layer with 36 neurons
layer1 = tflearn.fully_connected(input_data,39,activation='ReLU')
layer2 = tflearn.fully_connected(layer1,39,activation='relu6')
net = tflearn.fully_connected(layer1, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=300, batch_size=6, show_metric=True,validation_batch_size=0.2)
model.save('model.tflearn')

Training Step: 6299  | total loss: 0.40221 | time: 0.030s
| Adam | epoch: 300 | loss: 0.40221 - acc: 0.7945 -- iter: 120/121
Training Step: 6300  | total loss: 0.41322 | time: 0.032s
| Adam | epoch: 300 | loss: 0.41322 - acc: 0.7984 -- iter: 121/121
--


In [6]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )


In [7]:

trained_data = pickle.load( open( "training_data", "rb" ) )
words = trained_data['words']
classes = trained_data['classes']
train_x = trained_data['train_x']
train_y = trained_data['train_y']

In [8]:
# load our saved model
model.load('./model.tflearn')

W0705 16:54:03.702435   244 deprecation.py:323] From c:\program files\python37\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [9]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def make_vocab(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    #vocabulary of words
    vocab = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                vocab[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(vocab))

In [10]:
p = make_vocab("is your shop open today?", words)
print (p)

[0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [11]:
# ERROR_THRESHOLD = 0.25
# def classify(sentence):
#     # generate probabilities from the model
#     results = model.predict([unbag(sentence, words)])[0]
#     # filter out predictions below a threshold
#     results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
#     # sort by strength of probability
#     results.sort(key=lambda x: x[1], reverse=True)
#     return_list = []
#     for r in results:
#         return_list.append((classes[r[0]], r[1]))
#     # return tuple of intent and probability
#     return return_list

# def response(sentence:str, userID='123', show_details=False):
#     results = classify(sentence)
#     # if we have a classification then find the matching intent tag
#     if results:
#         # loop as long as there are matches to process
#         while results:
#             for i in intents['intents']:
#                 # find a tag matching the first result
#                 if i['tag'] == results[0][0]:
#                     # a random response from the intent
#                     return str(random.choice(i['responses']))
                

#             results.pop(0)

In [12]:
#classify('is your shop open today?')

In [13]:
#response('is your shop open today?')

In [14]:
#response('do you take cash?')

In [15]:
#response('what kind of mopeds do you rent?')

In [16]:
#response('Goodbye, see you later')

In [17]:
# create a data structure to hold user context
context = {}
#minimum probability we accept that the bot is sure of what it says, in this case bot must be at least 
#10% sure before it replies
ERROR_THRESHOLD = 0.10
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([make_vocab(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[k,v] for k,v in enumerate(results) if v>ERROR_THRESHOLD]
    # sort by strength of probability/descending order
    results.sort(key=lambda x: x[1], reverse=True)
    classify_result = []
    for result in results:
        classify_result.append((classes[result[0]], result[1]))
    # return tuple of intent and probability
    return classify_result

def response(sentence, userID='user', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for intent in intents['intents']:
                # find a tag matching the first result
                if intent['tag'] == results[0][0]:
                   #if the array json object contains a value of 'context_set', set that to the user session
                    if 'context_set' in intent:
                        if show_details: print ('context:', intent['context_set'])
                        context[userID] = intent['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in intent or \
                        (userID in context and 'context_filter' in intent and intent['context_filter'] == context[userID]):
                        if show_details: print ('tag:', intent['tag'])
                        # a random response from the intent
                        return str(random.choice(intent['responses']))

            results.pop(0)

In [18]:
response('we want to rent a moped')

'Click on start then move to control panel, are you there?'

In [19]:
response('yeah')

'Nice, now click overclock, and then finfinancials. Clear the cache and agree to reboot'

In [20]:
response('today')

"We're open every day from 9am-9pm"

In [21]:
response('today please')

'Our hours are 9am-9pm every day'

In [22]:
context

{'user': 'per_troubleshoot'}

In [23]:
response("Hi there!", show_details=True)

context: 
tag: greeting


'Hello, thanks for visiting Fintech'

In [24]:
response('today')

"We're open every day from 9am-9pm"

In [25]:
classify('today')

[('opentoday', 0.71860814), ('soft_issues_response_noantivirus', 0.12539037)]

In [26]:
response('hi')

'Hello, thanks for visiting Fintech'

In [27]:
response('i have some erors')

'does the log message displayed contain HARD or SOFT at the ending?'

In [28]:
classify("explain finfinancials?")

[('finfinancials', 0.9940521)]

In [29]:
context

{'user': 'error_finder'}

In [34]:
# Flask is a micro framework for Python aiming to provide the flexibility of creating web sites and APIs.
from flask import Flask, request,render_template
app = Flask(__name__)
app.static_folder = 'static'
@app.route('/')
def index():
	return render_template('index.html')
@app.route('/process',methods=['POST'])
def process():
	user_input=request.form['user_input']
	bot_response=response(user_input)
	if(bot_response is None):
		bot_response=random.choice(['Sorry, I fail to understand that, Im a bot','Please rephrase the question, im just a bot'])
    


	
	return render_template('index.html',user_input=user_input,bot_response=bot_response)
#make the session last longer as jupyter does not provide the ability
from werkzeug.serving import run_simple
run_simple('localhost', 9000, app)

I0705 16:56:53.276874   244 _internal.py:122]  * Running on http://localhost:9000/ (Press CTRL+C to quit)
I0705 16:56:55.726320   244 _internal.py:122] 127.0.0.1 - - [05/Jul/2019 16:56:55] "GET / HTTP/1.1" 200 -
I0705 16:56:58.736612   244 _internal.py:122] 127.0.0.1 - - [05/Jul/2019 16:56:58] "POST /process HTTP/1.1" 200 -
I0705 16:57:01.068172   244 _internal.py:122] 127.0.0.1 - - [05/Jul/2019 16:57:01] "POST /process HTTP/1.1" 200 -
I0705 16:57:01.368215   244 _internal.py:122] 127.0.0.1 - - [05/Jul/2019 16:57:01] "POST /process HTTP/1.1" 200 -
